### Imports

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import urllib
import matplotlib.dates as mdates

### Loading the data the dataframe

In [2]:
df = pd.read_csv('data.csv')
pd.options.display.max_columns = 999

### Reorganizing the data - Difficulty and Questions score
We want to change the 1/5 scale to -2/2 so that we can have 0 as an absolute value

In [3]:
df.iloc[:, 4:] = df.iloc[:, 4:].apply(lambda x: x - 3)

### Renaming the nb.repeats column name to no.taken 
The name is confusing and can be interpreted that a student has taken the calss already once and this
value represents the number of times that he repeated the class

In [4]:
df.rename(columns={'nb.repeat':'no.taken'}, inplace=True)

### Assessing the class size - Grouping the 13 classes

In [5]:
# spc means students per class
spc = df.groupby(['class']).size()
spc

class
1     303
2     140
3     904
4     187
5     656
6     558
7     187
8     500
9     571
10    448
11    484
12     41
13    841
dtype: int64

### Assessing how many students have taken the class and how many times

In [6]:
df.groupby('no.taken').size()

no.taken
1    4909
2     576
3     335
dtype: int64

In [7]:
# sapc means student attempts per class
sapc = df.groupby(['class', 'no.taken']).size()
sapc

class  no.taken
1      1           272
       2            25
       3             6
2      1           130
       2             7
       3             3
3      1           778
       2            87
       3            39
4      1           154
       2            20
       3            13
5      1           611
       2            28
       3            17
6      1           494
       2            55
       3             9
7      1           150
       2            14
       3            23
8      1           379
       2            69
       3            52
9      1           400
       2            99
       3            72
10     1           425
       2            14
       3             9
11     1           431
       2            43
       3            10
12     1            34
       2             5
       3             2
13     1           651
       2           110
       3            80
dtype: int64

### Converting the values in percentages to better understand the dataset

In [8]:
# sapc means student attempts per class percentage
sapcp = df.groupby(['class', 'no.taken']).size().reset_index(name='count')
sapcp['Percentage'] = sapcp.groupby('class')['count'].transform(lambda x: round(x/sum(x) * 100, 2))
sapcp

,class,no.taken,count,Percentage
0,1,1,272,89.77
1,1,2,25,8.25
2,1,3,6,1.98
3,2,1,130,92.86
4,2,2,7,5.00
5,2,3,3,2.14
6,3,1,778,86.06
7,3,2,87,9.62
8,3,3,39,4.31
9,4,1,154,82.35


### Getting the mean of the percentages from 1, 2, 3 attemps

In [9]:
sapcp.groupby('no.taken').mean()['Percentage']

no.taken
1    84.848462
2     9.508462
3     5.643077
Name: Percentage, dtype: float64

Based on the previous block, we can assume that the teachers had little to do with students passing/failing.
By calculating the mean for the atempts, we can see that 84.8% of students took the class once, 9.5% took the class twice and 5.6% three times 

### Split the data in chunks - classes to prepare it for further analysis
This way, we can quickly access the data by class

Data can be accessed like so dfdc[class]

In [10]:
# Create a list of classes
classes = df['class'].unique()
classes
# Create a data frame dictionary to store the entries per class for quick access
# dfd is dataframe dictionary
dfdc = {elem : pd.DataFrame for elem in classes}
for key in dfdc.keys():
    dfdc[key] = df[:][df['class'] == key]


### Split the data in chunks - teachers to prepare it for further analysis
This way, we can quickly access the entries relating to a teacher

Data can be accessed like so dfdt[teacher]

In [11]:
# Create a list of teachers
teachers = df['instr'].unique()
teachers
# Create a data frame dictionary to store the entries per teacher for quick access
# dfdp is dataframe dictionary
dfdp = {elem : pd.DataFrame for elem in teachers}
for key in dfdc.keys():
    dfdp[key] = df[:][df['instr'] == key]

### Observation: 

Based on our assumptions and analysis, we have noticed that some questions tend to repeat. That is because the author of the questionaire wanted to make sure that the students are answering almost the same to the questions at hand. After further analysis, we identified the following as potential duplicates:

- Q5 and Q7 - 263 (1)
- Q8 and Q10 - 251 (2)
- Q13 and Q14 - 102 (3)
- Q15 and Q19 - 250 (4)
- Q16 and Q18 - 191 (5)
- Q20 and Q25 - 233 (6) 
- Q21 and Q22 - 106 (7)
- Q24 and Q26 - 288 (8) 

The next steps are:
- Calculate the difference between the four groups of questions
- Assess how many of the entries have a difference greater than abs(1)
- Condense the duplicate questions if the percentage of such occurance is greater than 90%


In [12]:
# Create a dataframe that holds the questions and the differences between the ones that we think are duplicate
df_alt = df
df_alt['diff (1)'] = df_alt.apply(lambda x: x['Q5'] - x['Q7'], axis=1)
df_alt['diff (2)'] = df_alt.apply(lambda x: x['Q8'] - x['Q10'], axis=1)
df_alt['diff (3)'] = df_alt.apply(lambda x: x['Q13'] - x['Q14'], axis=1)
df_alt['diff (4)'] = df_alt.apply(lambda x: x['Q15'] - x['Q19'], axis=1)
df_alt['diff (5)'] = df_alt.apply(lambda x: x['Q16'] - x['Q18'], axis=1)
df_alt['diff (6)'] = df_alt.apply(lambda x: x['Q20'] - x['Q25'], axis=1)
df_alt['diff (7)'] = df_alt.apply(lambda x: x['Q21'] - x['Q22'], axis=1)
df_alt['diff (8)'] = df_alt.apply(lambda x: x['Q24'] - x['Q26'], axis=1)


In [13]:
# See how many records are have all the new columns greater than 1 or lower than -1 
# This will show us how many people have answered very differently to a duplicate question

# df_alt[((df_alt['diff (1)'] < -1) | (df_alt['diff (1)'] > 1))].count()
# df_alt[((df_alt['diff (2)'] < -1) | (df_alt['diff (2)'] > 1))].count()
# df_alt[((df_alt['diff (3)'] < -1) | (df_alt['diff (3)'] > 1))].count()
# df_alt[((df_alt['diff (4)'] < -1) | (df_alt['diff (4)'] > 1))].count()
# df_alt[((df_alt['diff (5)'] < -1) | (df_alt['diff (5)'] > 1))].count()
# df_alt[((df_alt['diff (6)'] < -1) | (df_alt['diff (6)'] > 1))].count()
# df_alt[((df_alt['diff (7)'] < -1) | (df_alt['diff (7)'] > 1))].count()
# df_alt[((df_alt['diff (8)'] < -1) | (df_alt['diff (8)'] > 1))].count()
df

,instr,class,no.taken,attendance,difficulty,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,Q27,Q28,diff (1),diff (2),diff (3),diff (4),diff (5),diff (6),diff (7),diff (8)
0,1,2,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,2,1,2,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,0,0,0,0,0,0,0,0
3,1,2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,2,1,0,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,0,0,0,0,0,0,0,0
5,1,2,1,3,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0
6,1,2,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0
7,1,2,1,1,0,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,0,0,0,0,0,0,0,0
8,1,2,1,1,0,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0
9,1,2,1,4,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0


### Entries that have a different answer greater than 1 or lower than -1 to a potential duplicated question
- Q5 and Q7 - 263 (1)
- Q8 and Q10 - 251 (2)
- Q13 and Q14 - 102 (3)
- Q15 and Q19 - 250 (4)
- Q16 and Q18 - 191 (5)
- Q20 and Q25 - 233 (6) 
- Q21 and Q22 - 106 (7)
- Q24 and Q26 - 288 (8) 

### Therefore we can mark the 8 groups as duplicate questions, eliminating the one that has the lower rating
- We can do this by making the average between a group of questions and eliminating one

In [14]:
df_alt['Q5 - Q7'] = df_alt.apply(lambda x: int(round((x['Q5'] + x['Q7']) / 2)), axis=1)
df_alt['Q8 - Q10'] = df_alt.apply(lambda x: int(round((x['Q8'] + x['Q10']) / 2)), axis=1)
df_alt['Q13 - Q14'] = df_alt.apply(lambda x: int(round((x['Q13'] + x['Q14']) / 2)), axis=1)
df_alt['Q15 - Q19'] = df_alt.apply(lambda x: int(round((x['Q15'] + x['Q19']) / 2)), axis=1)
df_alt['Q16 - Q18'] = df_alt.apply(lambda x: int(round((x['Q16'] + x['Q18']) / 2)), axis=1)
df_alt['Q20 - Q25'] = df_alt.apply(lambda x: int(round((x['Q20'] + x['Q25']) / 2)), axis=1)
df_alt['Q21 - Q22'] = df_alt.apply(lambda x: int(round((x['Q21'] + x['Q22']) / 2)), axis=1)
df_alt['Q24 - Q26'] = df_alt.apply(lambda x: int(round((x['Q24'] + x['Q26']) / 2)), axis=1)

### Confirming that the questions are close in score

In [131]:
df_alt[['Q5', 'Q7', 'Q8', 'Q10', 'Q13', 'Q14', 'Q15', 'Q19', 'Q16', 'Q18', 'Q20', 'Q25', 'Q21', 'Q22', 'Q24', 'Q26']].sum()

Q5      616
Q7      386
Q8      244
Q10     528
Q13    1413
Q14    1693
Q15    1672
Q19    1523
Q16     987
Q18    1295
Q20    1661
Q25    1819
Q21    1789
Q22    1848
Q24     971
Q26    1293
dtype: int64

### Eliminating the unwanted columns
This step is needed so that we can further analyse the data

In [15]:
df.drop(['Q5', 'Q7', 'Q8', 'Q10', 'Q13', 'Q14', 'Q15', 'Q19', 'Q16', 'Q18', 'Q20', 'Q25', 'Q21', 'Q22', 'Q24', 'Q26', 'diff (1)', 'diff (2)', 'diff (3)', 'diff (4)', 'diff (5)', 'diff (6)', 'diff (7)', 'diff (8)'], axis=1, inplace=True)
df

,instr,class,no.taken,attendance,difficulty,Q1,Q2,Q3,Q4,Q6,Q9,Q11,Q12,Q17,Q23,Q27,Q28,Q5 - Q7,Q8 - Q10,Q13 - Q14,Q15 - Q19,Q16 - Q18,Q20 - Q25,Q21 - Q22,Q24 - Q26
0,1,2,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,2,1,2,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
3,1,2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,2,1,0,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2
5,1,2,1,3,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
6,1,2,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
7,1,2,1,1,0,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
8,1,2,1,1,0,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1
9,1,2,1,4,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


### Grouping the dataset by themes - Chosen question that represent that theme
- Course Satisfaction - Q3, Q5-Q7, Q9, Q11, Q8-Q10 (5)
- Preparation/Organization - Q1, Q2, Q4, Q15-Q19, Q24-Q26 (5)
- Competence/Ethics - Q6, Q28, Q17, Q13-Q14, Q16-Q18(Because he is competent and has ethics he is committed to the course. I think)
- Helpfulness/Commitment - Q12, Q27, Q23, Q21-Q22, Q20-Q25 (5)

In [22]:
df_cs = df_alt[['instr', 'class', 'no.taken', 'attendance', 'difficulty', 'Q3', 'Q5 - Q7', 'Q9', 'Q11', 'Q8 - Q10']]
df_po = df_alt[['instr', 'class', 'no.taken', 'attendance', 'difficulty', 'Q1', 'Q2', 'Q4', 'Q15 - Q19', 'Q24 - Q26']]
df_ce = df_alt[['instr', 'class', 'no.taken', 'attendance', 'difficulty', 'Q6', 'Q13 - Q14', 'Q16 - Q18', 'Q17', 'Q28']]
df_hc = df_alt[['instr', 'class', 'no.taken', 'attendance', 'difficulty', 'Q12', 'Q20 - Q25', 'Q21 - Q22', 'Q23', 'Q27']]

In [31]:
# this is interesting - have to figure out what it all means.
df_alt.groupby(['instr', 'class']).describe()

no.taken                                              attendance  \
               count      mean       std  min  25%  50%  75%  max      count   
instr class                                                                    
1     2        140.0  1.092857  0.357779  1.0  1.0  1.0  1.0  3.0      140.0   
      7        187.0  1.320856  0.682928  1.0  1.0  1.0  1.0  3.0      187.0   
      10       448.0  1.071429  0.326716  1.0  1.0  1.0  1.0  3.0      448.0   
2     1        303.0  1.122112  0.383785  1.0  1.0  1.0  1.0  3.0      303.0   
      6        558.0  1.130824  0.382400  1.0  1.0  1.0  1.0  3.0      558.0   
      11       484.0  1.130165  0.393528  1.0  1.0  1.0  1.0  3.0      484.0   
      13        99.0  1.070707  0.327406  1.0  1.0  1.0  1.0  3.0       99.0   
3     3        904.0  1.182522  0.485543  1.0  1.0  1.0  1.0  3.0      904.0   
      4        187.0  1.245989  0.571192  1.0  1.0  1.0  1.0  3.0      187.0   
      5        656.0  1.094512  0.370970  1.0  1.0  1.0  1.0  3.0      656.0   
      8        500.0  1.346000  0.659662  1.0  1.0  1.0  1.0  3.0      500.0   
      9        571.0  1.425569  0.705351  1.0  1.0  1.0  2.0  3.0      571.0   
      12        41.0  1.219512  0.524985  1.0  1.0  1.0  1.0  3.0       41.0   
      13       742.0  1.354447  0.663061  1.0  1.0  1.0  1.0  3.0      742.0   

                                                         difficulty            \
                 mean       std  min  25%  50%  75%  max      count      mean   
instr class                                                                     
1     2      2.071429  1.284343  0.0  1.0  2.0  3.0  4.0      140.0  0.442857   
      7      2.037433  1.434479  0.0  1.0  2.0  3.0  4.0      187.0  0.823529   
      10     1.959821  1.458813  0.0  1.0  2.0  3.0  4.0      448.0 -0.321429   
2     1      1.749175  1.512571  0.0  0.0  1.0  3.0  4.0      303.0 -0.511551   
      6      1.758065  1.409342  0.0  0.0  2.0  3.0  4.0      558.0 -0.397849   
      11     1.876033  1.437851  0.0  1.0  2.0  3.0  4.0      484.0  0.018595   
      13     1.626263  1.562219  0.0  0.0  2.0  3.0  4.0       99.0 -0.959596   
3     3      1.475664  1.457962  0.0  0.0  1.0  3.0  4.0      904.0 -0.196903   
      4      1.433155  1.429178  0.0  0.0  1.0  3.0  4.0      187.0 -0.352941   
      5      1.742378  1.462750  0.0  0.0  2.0  3.0  4.0      656.0 -0.099085   
      8      1.982000  1.580719  0.0  0.0  2.0  3.0  4.0      500.0  0.000000   
      9      1.427320  1.442302  0.0  0.0  1.0  3.0  4.0      571.0 -0.346760   
      12     1.707317  1.289261  0.0  1.0  1.0  3.0  4.0       41.0 -0.097561   
      13     1.350404  1.442976  0.0  0.0  1.0  3.0  4.0      742.0 -0.482480   

                                                   Q1                      \
                  std  min  25%  50%  75%  max  count      mean       std   
instr class                                                                 
1     2      1.081376 -2.0  0.0  1.0  1.0  2.0  140.0  0.421429  1.449723   
      7      1.050285 -2.0  0.0  1.0  2.0  2.0  187.0 -0.272727  1.293264   
      10     1.190674 -2.0 -1.0  0.0  0.0  2.0  448.0  0.361607  1.279174   
2     1      1.124250 -2.0 -2.0  0.0  0.0  2.0  303.0  0.171617  1.290664   
      6      1.096491 -2.0 -1.0  0.0  0.0  2.0  558.0  0.098566  1.320722   
      11     1.199407 -2.0 -1.0  0.0  1.0  2.0  484.0  0.148760  1.284797   
      13     1.133166 -2.0 -2.0 -1.0  0.0  2.0   99.0 -0.060606  1.361408   
3     3      1.546958 -2.0 -2.0  0.0  1.0  2.0  904.0 -0.171460  1.309487   
      4      1.434419 -2.0 -2.0  0.0  1.0  2.0  187.0 -0.566845  1.195679   
      5      1.456001 -2.0 -2.0  0.0  1.0  2.0  656.0  0.030488  1.292013   
      8      1.270917 -2.0 -1.0  0.0  1.0  2.0  500.0 -0.308000  1.344899   
      9      1.308100 -2.0 -2.0  0.0  1.0  2.0  571.0 -0.150613  1.307903   
      12     0.969662 -2.0  0.0  0.0  0.0  2.0   41.0 -0.219512  1.313625   
      13     1.409086 -2.0 -2.0 -1.0  1.0

# Work in progress. At least the data is cleaned and ready for analysis/plots

### Important observation

It seems that there are a lot of entries that have the same score in all the question

This might mean that the students have untruthfully graded the questions due to laziness or other factors

Though the factors are not important, we want to assess what is the percentage of entries that have the difference between the entry itself and the mode of the scores of the question 0. At a later date we will determine whether such entries should be treated differently or not.

In [16]:
# First step is to create a dataset that holds only the questions
# qdf represents questions dataframe
qdf = df[df.columns.difference(['instr', 'class', 'no.taken', 'attendance', 'difficulty'])]
qdf

,Q1,Q11,Q12,Q13 - Q14,Q15 - Q19,Q16 - Q18,Q17,Q2,Q20 - Q25,Q21 - Q22,Q23,Q24 - Q26,Q27,Q28,Q3,Q4,Q5 - Q7,Q6,Q8 - Q10,Q9
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2
5,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
6,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
7,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
8,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
9,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [17]:
# Second step is to calculate the median of the questions per row
qdf['median'] = qdf.median(1)
qdf

/Users/_bogz_/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Q1,Q11,Q12,Q13 - Q14,Q15 - Q19,Q16 - Q18,Q17,Q2,Q20 - Q25,Q21 - Q22,Q23,Q24 - Q26,Q27,Q28,Q3,Q4,Q5 - Q7,Q6,Q8 - Q10,Q9,median
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2.0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
4,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2.0
5,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1.0
6,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1.0
7,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2.0
8,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1.0
9,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1.0


In [18]:
# Third step is to make the difference between each value of the questions and the median
qdf = qdf.apply(lambda x: x - x['median'], axis=1)
qdf

,Q1,Q11,Q12,Q13 - Q14,Q15 - Q19,Q16 - Q18,Q17,Q2,Q20 - Q25,Q21 - Q22,Q23,Q24 - Q26,Q27,Q28,Q3,Q4,Q5 - Q7,Q6,Q8 - Q10,Q9,median
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
df_alt

,instr,class,no.taken,attendance,difficulty,Q1,Q2,Q3,Q4,Q6,Q9,Q11,Q12,Q17,Q23,Q27,Q28,Q5 - Q7,Q8 - Q10,Q13 - Q14,Q15 - Q19,Q16 - Q18,Q20 - Q25,Q21 - Q22,Q24 - Q26
0,1,2,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,2,1,2,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
3,1,2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,2,1,0,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2
5,1,2,1,3,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
6,1,2,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
7,1,2,1,1,0,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
8,1,2,1,1,0,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1
9,1,2,1,4,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [75]:
100 *2985/5819 

51.297473792747894

In [ ]:
## 51% of the values are all equal. What do we do now...